In [23]:
!pip install requests selenium

In [ ]:
import os
import csv
import time
import json
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
import os
import json
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    return webdriver.Chrome(options=options)

def click_tab(driver, tab_text):
    tabs = driver.find_elements(By.XPATH, f"//a[span[contains(text(), '{tab_text}')]]")
    for tab in tabs:
        if tab.is_displayed():
            tab.click()
            time.sleep(2)
            return
    print(f"[WARN] Tab with text '{tab_text}' not found.")

def scrape_residential_tab(driver):
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "Residential")))
    except:
        print("[WARN] Residential table not found.")
        return {}

    data = {}
    rows = driver.find_elements(By.CSS_SELECTOR, "#Residential tr")
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) >= 2:
            key = cells[0].text.strip().replace(":", "")
            value = cells[1].text.strip()
            if key:
                data[key] = value
    return data

def wait_for_photoDetails(driver, timeout=10):
    for _ in range(timeout * 10):
        try:
            result = driver.execute_script("return typeof photoDetails !== 'undefined' && photoDetails.length > 0")
            if result:
                return True
        except:
            pass
        time.sleep(0.1)
    return False

def scrape_and_download_photos_from_photoDetails(driver, address_folder):
    if not wait_for_photoDetails(driver):
        print("[WARN] photoDetails not found or empty.")
        return []

    try:
        photo_details_json = driver.execute_script("return JSON.stringify(photoDetails);")
        photo_details = json.loads(photo_details_json)
    except Exception as e:
        print(f"[ERROR] Couldn't extract photoDetails: {e}")
        return []

    photo_urls = []
    for idx, photo in enumerate(photo_details):
        std_url = f"https://www.ncpub.org/_web/api/document/{photo['Id']}/standard?token=RnNBOFBQNFhzakRDS3dzVVFPYm1wVHpMMFhZR2FvVGZSWEFmRkc5SDE0az0="
        photo_urls.append(std_url)
        try:
            img_data = requests.get(std_url).content
            with open(os.path.join(address_folder, f"photo_{idx+1}.jpg"), 'wb') as f:
                f.write(img_data)
        except Exception as e:
            print(f"[WARN] Failed to download image {idx+1}: {e}")
    return photo_urls

def get_total_record_count(driver):
    try:
        txt = driver.find_element(By.ID, "DTLNavigator_txtFromTo").get_attribute("value")
        return int(txt.split(" of ")[-1])
    except:
        return 1

def scrape_all_records_on_street(driver, street_name, output_dir):
    base_url = "https://www.ncpub.org/_web/search/commonsearch.aspx?mode=address"
    driver.get(base_url)
    time.sleep(2)

    try:
        driver.find_element(By.ID, "btAgree").click()
        time.sleep(2)
    except:
        pass

    driver.find_element(By.ID, "inpStreet").send_keys(street_name)
    driver.find_element(By.ID, "btSearch").click()
    time.sleep(3)

    # Click the first result
    try:
        result_links = driver.find_elements(By.CSS_SELECTOR, "#searchResults tr.SearchResults")
        if not result_links:
            print(f"[INFO] No results found for street: {street_name}")
            return
        result_links[0].click()
        time.sleep(2)
    except Exception as e:
        print(f"[ERROR] Could not click initial search result: {e}")
        return

    total = get_total_record_count(driver)
    print(f"[INFO] Found {total} records for {street_name}")

    for i in range(total):
        try:
            click_tab(driver, "Residential")
            data = scrape_residential_tab(driver)

            click_tab(driver, "Photos")
            folder = os.path.join(output_dir, data.get("address", f"{street_name}_{i}").replace(" ", "_"))
            os.makedirs(folder, exist_ok=True)
            photo_urls = scrape_and_download_photos_from_photoDetails(driver, folder)

            data["photo_urls"] = photo_urls

            out_json = os.path.join(folder, "data.json")
            with open(out_json, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2)

            print(f"[SAVED] {data.get('address', 'Unknown')}")

            if i < total - 1:
                driver.find_element(By.ID, "DTLNavigator_imageNext").click()
                time.sleep(2)
        except Exception as e:
            print(f"[WARN] Skipped record {i+1} due to error: {e}")
            break

In [ ]:
driver = init_driver(headless=False)
output_dir = "scraped_properties"
os.makedirs(output_dir, exist_ok=True)

STREETS = ["IRONSTONE RD", "IRONSTONE CT", "STANBRIDGE CT", "HIGHBRIDGE CT", "TUDOR CT", "SUTTON PL", "REGAL RD", "GRAMERCY PL", "MARGATE RD"]

for street in STREETS:
  scrape_all_records_on_street(driver, street, output_dir)

driver.quit()

[INFO] Found 11 records for FORNANCE RD
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[INFO] Found 29 records for NIJARO RD
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[INFO] Found 18 records for LENAIRE RD
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SAVED] Unknown
[SA

InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x000000010087e668 cxxbridge1$str$ptr + 2723108
1   chromedriver                        0x00000001008768dc cxxbridge1$str$ptr + 2690968
2   chromedriver                        0x00000001003ca588 cxxbridge1$string$len + 90032
3   chromedriver                        0x0000000100404e8c cxxbridge1$string$len + 329908
4   chromedriver                        0x000000010042d578 cxxbridge1$string$len + 495520
5   chromedriver                        0x000000010042c880 cxxbridge1$string$len + 492200
6   chromedriver                        0x000000010039aaf0 chromedriver + 92912
7   chromedriver                        0x000000010084228c cxxbridge1$str$ptr + 2476360
8   chromedriver                        0x0000000100845520 cxxbridge1$str$ptr + 2489308
9   chromedriver                        0x0000000100823a78 cxxbridge1$str$ptr + 2351412
10  chromedriver                        0x000000010084